In [1]:
import numpy as np
import pandas as pd
import math
import sklearn
import sklearn.preprocessing
import datetime
pd.options.mode.chained_assignment = None  

In [2]:
#import dataframe
df = pd.read_csv(r"C:\Users\krist\Documents\Data\ger_factor_data_short_copy.csv", dtype ={"comp_tpci": str})
df["eom"] = pd.to_datetime(df["eom"])    #convert to date format
df.head()

,id,eom,prc,me,ret,ret_exc,ret_exc_lead1m,niq_su,ret_6_1,ret_12_1,...,betabab_1260d,noa_at,mispricing_mgmt,seas_6_10na,cop_atl1,prc_highprc_252d,ocf_at,saleq_su,dbnetis_at,netdebt_me
0,comp_001166_02W,1999-07-31,7.492072,265.151983,0.068365,0.064565,-0.066388,-0.211288,NaN,NaN,...,NaN,0.607343,NaN,NaN,0.200825,NaN,-0.047722,-0.797858,-0.03544,0.519617
1,comp_001166_02W,1999-08-31,7.023910,248.583248,-0.062488,-0.066388,0.178763,-0.211288,NaN,NaN,...,NaN,0.607343,NaN,NaN,0.200825,NaN,-0.047722,-0.797858,-0.03544,0.553442
2,comp_001166_02W,1999-09-30,8.306919,293.990256,0.182663,0.178763,-0.029641,-0.211288,NaN,NaN,...,NaN,0.607343,NaN,NaN,0.200825,NaN,-0.047722,-0.797858,-0.03544,0.469985
3,comp_001166_02W,1999-10-31,8.093089,302.159123,-0.025741,-0.029641,0.974577,0.157763,NaN,NaN,...,NaN,0.659876,NaN,NaN,0.321979,NaN,0.093191,1.315548,-0.03908,0.444769
4,comp_001166_02W,1999-11-30,16.009565,634.008234,0.978177,0.974577,0.481106,0.157763,NaN,NaN,...,NaN,0.659876,NaN,NaN,0.321979,NaN,0.093191,1.315548,-0.03908,0.212792


### Filter out penny stocks

The data sample is limited to large German stocks with a market capitalisation of at least USD 25 million and a minimum share price of USD 5, thus excluding penny stocks

In [23]:
# Drop all observations with stock price < 5 USD:
df = df.loc[df["prc"] >= 5]
# Drop all observations where the market capitalisation < 25mio USD:
df = df.loc[df["me"] >= 25]

### Treating missing values

Only the independent variables with less than 25 per cent missing values are retained. The remaining missing values are replaced with the with the cross-sectional median at each month for each stock in line with Gu, Kelly, and Xiu (2020).

- Drop observations where the variable of interest is missing

In [4]:
df = df.dropna( how='any',subset=["ret_exc_lead1m"])

- Inspect variables with the most missing values:

In [5]:
pd.set_option('display.max_rows',None)
df.isna().sum().sort_values(ascending = False)

seas_6_10na         177345
rd_me               145890
rd_sale             137350
saleq_su            123755
ocfq_saleq_std      118058
ni_inc8q            112203
dsale_dsga          103839
niq_su              103127
capex_abn            93484
mispricing_mgmt      91853
zero_trades_252d     84636
qmj_prof             84116
ami_126d             83213
turnover_126d        81295
dolvol_126d          81249
niq_at               80134
div12m_me            78953
betabab_1260d        77618
fcf_me               77383
capx_gr1a            70693
emp_gr1.1            61633
emp_gr1              61633
dsale_dinv           57218
ni_ivol              55833
f_score              54562
beta_60m             53243
dsale_drec           52983
taccruals_ni         42464
cowc_gr1a            42250
noa_at               42019
cop_atl1             41654
taccruals_at         41366
oaccruals_at         40256
gp_bev               39453
ca_cl                34859
caliq_cl             34841
sale_gr3             32946
p

- Drop the variables with more than 25% missing values

In [7]:
limitPer = len(df) * .75
df = df.dropna(thresh=limitPer, axis=1)

- Inspect remaining variables:

In [8]:
pd.set_option('display.max_rows',None)
df.isna().sum().sort_values(ascending = False)

f_score             54562
beta_60m            53243
dsale_drec          52983
taccruals_ni        42464
cowc_gr1a           42250
noa_at              42019
cop_atl1            41654
taccruals_at        41366
oaccruals_at        40256
gp_bev              39453
ca_cl               34859
caliq_cl            34841
sale_gr3            32946
pi_nix              32246
rmax1_21d           29653
rvol_21d            29653
debt_gr1            28724
ret_12_7            28372
ret_12_1            28359
lnoa_gr1a           27782
debt_me             24688
netdebt_me          24688
ret_9_1             24302
ivol_capm_252d      24047
prc_highprc_252d    24047
be_me               23607
sale_me             22505
bidaskhl_21d        22222
ni_me               22112
ocf_debt            22001
ret_6_1             20093
ret_6_0             19796
inv_gr1a            19160
cash_gr1a           17535
chcsho_12m          17410
debtlt_gr1a         14295
ret_1_0             11708
be_gr1a             11036
sale_gr1    

- Replace the missing valuues with the cross-sectional median at each month


In [9]:
df = df.fillna(df.groupby('eom').transform('median'))

-  Drop all observations for which there are still missing values

In [10]:
df.dropna(inplace=True)

- There are 214,016 observations left

In [11]:
print(len(df))

214016


In [12]:
pd.set_option('display.max_rows',None)
df.isna().sum().sort_values(ascending = False)

id                  0
dsale_drec          0
ni_be               0
ocf_debt            0
cash_gr1a           0
at_gr1              0
chcsho_12m          0
ca_cl               0
ivol_capm_252d      0
debt_gr1            0
debtlt_gr1a         0
ope_be              0
caliq_cl            0
f_score             0
ret_9_1             0
rvol_21d            0
gp_bev              0
cowc_gr1a           0
pi_nix              0
ret_6_0             0
ret_1_0             0
noa_at              0
cop_atl1            0
prc_highprc_252d    0
ocf_at              0
dbnetis_at          0
ret_12_7            0
bidaskhl_21d        0
eom                 0
sale_gr3            0
prc                 0
me                  0
ret                 0
ret_exc             0
ret_exc_lead1m      0
ret_6_1             0
ret_12_1            0
tax_gr1a            0
be_me               0
debt_me             0
ni_me               0
sale_gr1            0
rmax1_21d           0
sale_me             0
lnoa_gr1a           0
inv_gr1a  

## Explore the data before treating 


In [13]:
df = df.sort_values(by=['eom', "id"])

In [16]:
#start from 2003--> first year with complete data for all months
df = df[~(df['eom'] < '2003-01-31')]

In [17]:
#the new observation period contains 18 years (2016 Months) form 01.2003 to 12.2020 
print(df['eom'].nunique())

216


In [18]:
#There are 1855 unique german stocks
print(df['id'].nunique())

1855


In [19]:
#Inspect variable types
df.info(verbose=True)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 156948 entries, 42 to 360494
Data columns (total 55 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   id                156948 non-null  object        
 1   eom               156948 non-null  datetime64[ns]
 2   prc               156948 non-null  float64       
 3   me                156948 non-null  float64       
 4   ret               156948 non-null  float64       
 5   ret_exc           156948 non-null  float64       
 6   ret_exc_lead1m    156948 non-null  float64       
 7   ret_6_1           156948 non-null  float64       
 8   ret_12_1          156948 non-null  float64       
 9   tax_gr1a          156948 non-null  float64       
 10  be_me             156948 non-null  float64       
 11  debt_me           156948 non-null  float64       
 12  ni_me             156948 non-null  float64       
 13  sale_gr3          156948 non-null  float64       
 14  sal

### Treating outliers

All monthly firm characteristics are winsorized at the 1% and 99% levels to ensure that the results are insensitive to outliers. In contrast to Gu, Kelly, and Xiu (2020), the dependent variable is also winsorized

In [134]:
variables = df.columns[~df.columns.isin(["eom",'id'])].tolist()
df[variables] = df[variables].apply(lambda x: x.clip(*x.quantile([0.01, 0.99])))

In [135]:
#Save dataset:
df.to_csv(r'C:\Users\krist\Documents\Data\ger_factor_data_from2003.csv', index = False)